In [1]:
import pandas as pd

In [4]:
#Create a column filtering list
col_num = [0]
col_num.extend(list(range(1, 63, 2)))

#Column names
list_name = ['name']
for i in range(1, 10):
    list_name.extend(['0{}'.format(i)])
for i in range(10, 32):
    list_name.extend(['{}'.format(i)])   
#Read original data
data = pd.read_excel('../AnalysisData/weather_importable.xlsx', index_col=None, header=None, names=list_name, usecols=col_num)

#Replace -9999 with NA
data.replace(-9999, pd.NA, inplace=True)

#Separation of information in the first column
for i in range(0,data.shape[0]):
    first_col = data['name'][i]
    new_col = first_col[0:11]+' '+first_col[11:15]+' '+first_col[15:17]+' '+first_col[17:21]
    data['name'][i] = new_col
data[['id','year','month','elem']] = data.name.str.split(' ', expand=True)
new_list_name = ['id','year','month','elem']
new_list_name.extend(list_name[1:])
data = data[new_list_name]

#Data filtration
data = data[data['year'] == '2010']
data = data[data['month'] <= '05']
data.set_axis(list(range(1,16)), axis=0, inplace=True)

#Melt
data = data.melt(id_vars=['id','year','month','elem'])

#Data tidy
data = data[(pd.notna(data['value']))]
data.set_axis(list(range(1,data.shape[0]+1)), axis=0, inplace=True)
data['date'] = data['year']+'-'+data['month']+'-'+(data['variable'])
data = data[['id','date','elem','value']]

#Prcp data tidy
prcp_data = data[data['elem'] == 'PRCP']
prcp_data = prcp_data[prcp_data['value'] > 0]
prcp_data.set_axis(list(range(1,prcp_data.shape[0]+1)), axis=0, inplace=True)
prcp_data = prcp_data[['id','date','value']]
prcp_data = prcp_data.rename(columns={"value": "prcp"})
#Prcp data tidy - repair value
for i in range(1,prcp_data.shape[0]+1):
    val = prcp_data['prcp'][i]
    val = str(val)
    prcp_data['prcp'][i] = val[:-1]+'.'+val[-1:]

#Temp data tidy
temp_data = data[data['elem'] != 'PRCP']
temp_data.set_axis(list(range(1,temp_data.shape[0]+1)), axis=0, inplace=True)
#Temp data tidy - repair value
for i in range(1,temp_data.shape[0]+1):
    val = temp_data['value'][i]
    val = str(val)
    temp_data['value'][i] = val[:-1]+'.'+val[-1:]
#Temp data tidy - separation of columns
temp_data = temp_data[['id','date','value']]
temp_data['value_2'] = temp_data['value']
for i in range(1,temp_data.shape[0]):
    temp_data['value_2'][i] = temp_data['value_2'][i+1]
temp_data = temp_data.drop(list(range(2, temp_data.shape[0]+1, 2)), axis=0)
temp_data = temp_data.rename(columns={"value": "tmax", "value_2": "tmin"})
temp_data['date'] = pd.to_datetime(temp_data.date)
temp_data['date'] = temp_data['date'].dt.date
temp_data.sort_values(by='date', inplace=True)
temp_data.set_axis(list(range(1,temp_data.shape[0]+1)), axis=0, inplace=True)

#Saving data to file
temp_data.to_excel(excel_writer="../AnalysisData/temp_data.xlsx", index=False)
prcp_data.to_excel(excel_writer="../AnalysisData/prcp_data.xlsx", index=False)

#Set title
prcp_data = prcp_data.style.set_caption("The total liquid water equivalent of presumably all precipitation on the observed days in the period 01.2010-05.2010:")
temp_data = temp_data.style.set_caption("The maximum and minimum temperatures on the observed days in the period 01.2010-05.2010:")

In [5]:
prcp_data

,id,date,prcp
1,MX000017004,2010-05-29,2.5


In [6]:
temp_data

,id,date,tmax,tmin
1,MX000017004,2010-01-30,27.8,14.5
2,MX000017004,2010-02-02,27.3,14.4
3,MX000017004,2010-02-03,24.1,14.4
4,MX000017004,2010-02-11,29.7,13.4
5,MX000017004,2010-02-23,29.9,10.7
6,MX000017004,2010-03-05,32.1,14.2
7,MX000017004,2010-03-10,34.5,16.8
8,MX000017004,2010-03-16,31.1,17.6
9,MX000017004,2010-04-27,36.3,16.7
10,MX000017004,2010-05-27,33.2,18.2
